## 用Keras Tuner实现超参数搜索

安装keras-tuner:  
```shell
pip install -q -U keras-tuner
```

In [1]:
import sys
import os
import IPython

import tensorflow as tf
from tensorflow import keras
import kerastuner as kt

print("python version:", sys.version_info)
for module in tf, keras, kt:
    print(module.__name__, "version:", module.__version__)

python version: sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
tensorflow version: 2.3.0
tensorflow.keras version: 2.4.0
kerastuner version: 1.0.1


### 1. 加载和准备数据集

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

### 2. 定义模型

当构建用于超调整的模型时，除了模型体系结构之外，您还定义了超参数搜索空间。 您为超调优设置的模型称为超模型。  
  
您可以通过两种方法定义超模型：  
* 通过使用模型构建器功能  
* 通过子类化Keras Tuner API的HyperModel类  
  
您还可以使用两个预定义的HyperModel类-HyperXception和HyperResNet用于计算机视觉应用程序。  
  
在本教程中，您将使用模型构建器功能来定义图像分类模型。 模型构建器函数返回已编译的模型，并使用您内联定义的超参数对模型进行超调。  

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    
    hp_units = hp.Int("units", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))
    
    hp_learning_rate=hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=["accuracy"])
    return model

### 3.tuner搜索超参数

实例化调谐器以执行超调谐。  Keras调谐器有四个可用的调谐器-RandomSearch，Hyperband，BayesianOptimization和Sklearn。 在本教程中，您将使用Hyperband调谐器。
  
要实例化Hyperband调谐器，必须指定超模型，要优化的目标以及要训练的最大时期数（max_epochs）。

In [5]:
tuner = kt.Hyperband(
    model_builder,
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="my_dir",
    project_name="intro_to_kt",
)

 Hyperband 调整算法使用自适应资源分配和提前停止来快速收敛到高性能模型上。 这是使用运动冠军风格的支架完成的。   
该算法在几个 epoch 内训练了大量模型，并且仅将性能最高的一半模型进行到下一轮。 Hyperband 通过计算 $1 + log_{factor}(max\_epochs)$ 并将其四舍五入到最接近的整数来确定要在括号中训练的模型的数量。

 
在运行超参数搜索之前，定义一个回调以在每个训练步骤结束时清除训练输出。

In [6]:
class ClearTrainingOutput(keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)

运行超参数搜索。 除了上面的回调外，搜索方法的参数与 keras.model.fit 所使用的参数相同。

In [7]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.001.



要完成本教程，请使用搜索中的最佳超参数重新训练模型。

In [8]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data=(img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - ETA: 0s - loss: 2.4455 - accuracy: 0.06 - ETA: 4s - loss: 1.3105 - accuracy: 0.58 - ETA: 4s - loss: 1.1105 - accuracy: 0.63 - ETA: 4s - loss: 0.9879 - accuracy: 0.66 - ETA: 4s - loss: 0.9036 - accuracy: 0.69 - ETA: 4s - loss: 0.8595 - accuracy: 0.70 - ETA: 4s - loss: 0.8326 - accuracy: 0.71 - ETA: 4s - loss: 0.8054 - accuracy: 0.72 - ETA: 4s - loss: 0.7769 - accuracy: 0.73 - ETA: 3s - loss: 0.7497 - accuracy: 0.74 - ETA: 3s - loss: 0.7266 - accuracy: 0.74 - ETA: 3s - loss: 0.7108 - accuracy: 0.75 - ETA: 3s - loss: 0.6898 - accuracy: 0.76 - ETA: 3s - loss: 0.6775 - accuracy: 0.76 - ETA: 3s - loss: 0.6660 - accuracy: 0.76 - ETA: 3s - loss: 0.6542 - accuracy: 0.77 - ETA: 3s - loss: 0.6454 - accuracy: 0.77 - ETA: 3s - loss: 0.6378 - accuracy: 0.77 - ETA: 3s - loss: 0.6296 - accuracy: 0.78 - ETA: 3s - loss: 0.6256 - accuracy: 0.78 - ETA: 3s - loss: 0.6156 - accuracy: 0.78 - ETA: 3s - loss: 0.6108 - accuracy: 0.78 - ETA: 3s - loss: 0.60

my_dir / intro_to_kt目录包含在超参数搜索期间运行的每个试验（模型配置）的详细日志和checkpoints。   
如果重新运行超参数搜索，Keras Tuner将使用这些日志中的现有状态来继续搜索。 要禁用此行为，请在实例化调谐器时传递一个额外的overwrite = True参数。    